## 7324 Assignment A6 : NN and PCA
##### Name: Thang Nguyen
##### SMU ID: 48689334

## Imports

In [1]:
# The Classifiers
from keras.models import Sequential
from keras.layers import Dense
from sklearn.decomposition import PCA
# Data Wrangling Tool
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Foundational libraries
import pandas as pd
import numpy as np

## Loading Dataset

In [2]:
# note: I added the column names manually into the csv prior
# original csv from UCI comes without columns
abalone_df = pd.read_csv("../data/abalone.csv")
# checking dataframe
abalone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [3]:
abalone_df.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Cleaning Dataset

### One-hot Encoding abalone sex

In [4]:
abalone_df = pd.concat([abalone_df, pd.get_dummies(abalone_df['Sex'], prefix = 'Sex')], axis = 1)
abalone_df.drop(['Sex'], axis = 1, inplace = True)

In [5]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0


### Categorizing Rings into groups where
#### 0-4 -> infant
#### 5-9 -> young
#### 10-14 -> teen
#### 15-20 -> young adult
#### 20-25 -> adult

In [6]:
def group_rings(rings):
    if rings <= 4:
        return 'infant'
    if rings <= 9:
        return 'young'
    if rings <= 14:
        return 'teen'
    if rings <= 20: 
        return 'young adult'
    if rings <= 25:
        return 'adult'

In [7]:
abalone_df["Rings"] = abalone_df["Rings"].apply(group_rings)

In [8]:
abalone_df.head()

,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,young adult,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,young,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,young,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,teen,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,young,0,1,0


## Analysis